# Applied Data Science Capstone
## Week 3 -> Segementing and Clustering Neighbourhoods in Toronto

# Part 1

In [23]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from geopy.geocoders import Nominatim

### Downloading data from web saving in a dataframe

In [24]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=df[0]
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing the cells where Borough is not assigned

In [25]:
df=df[~(df['Borough']=='Not assigned')].reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [26]:
df_cleaned=pd.DataFrame([]) # Dataframe to save post processed data

### Cleaning data to remove the Non assigned Boroughs and adding multiples neighbourhoods in a signle string

In [27]:
df_cleaned['Postcode']=df['Postcode'].unique()
df_cleaned.set_index('Postcode',inplace=True)
df_grouped=df.groupby('Postcode')
for (postcode,group) in df_grouped:
    df_cleaned.loc[postcode,'Borough']=     ', '.join(set(group.Borough))
    df_cleaned.loc[postcode,'Neighborhood']=', '.join(set(group.Neighborhood))
    if df_cleaned.loc[postcode,'Neighborhood']=='Not assigned':
        df_cleaned.loc[postcode,'Neighborhood']=df_cleaned.loc[postcode,'Borough']
df_cleaned.reset_index(drop=False, inplace=True)
df_cleaned.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [28]:
df_cleaned.shape

(103, 3)

# Part 2

### Merging the clearned data with the geospacial data

In [29]:
geospacial=pd.read_csv('http://cocl.us/Geospatial_data')
Merged_df=pd.merge(df_cleaned,geospacial,left_on='Postcode',right_on='Postal Code')
Merged_df.drop('Postal Code',inplace=True,axis=1)

In [30]:
Merged_df.head(5)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Part 3

## Exploring the clustering the neighborhoods in Toronto

In [31]:
# Filtering Boroughs that have the word Toronto in them
Toronto_data=Merged_df[Merged_df['Borough'].str.lower().str.find('toronto')>0]
Toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [40]:
Toronto_data.shape

(39, 5)

# Define Foursquare Credentials and Version

In [32]:
CLIENT_ID = 'YT0EDB1LIXWATJQOLVZ1PSQMSN1PNAGZGZCP5SCSMYRWHMIR' # your Foursquare ID
CLIENT_SECRET = 'ZIVSV1Q1ZNU2MC1TAHBLKAXBOPATECKAUZCFVJG3F3UGZBJM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YT0EDB1LIXWATJQOLVZ1PSQMSN1PNAGZGZCP5SCSMYRWHMIR
CLIENT_SECRET:ZIVSV1Q1ZNU2MC1TAHBLKAXBOPATECKAUZCFVJG3F3UGZBJM


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #nearby_venues = pd.DataFrame([item for venue_list in venues_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Harbourfront
47
Queen's Park
0
Garden District, Ryerson
100
St. James Town
100
The Beaches
5
Berczy Park
56
Central Bay Street
82
Christie
16
Richmond, King, Adelaide
100
Dufferin, Dovercourt Village
16
Union Station, Toronto Islands, Harbourfront East
100
Little Portugal, Trinity
55
Riverdale, The Danforth West
42
Toronto Dominion Centre, Design Exchange
100
Exhibition Place, Brockton, Parkdale Village
21
India Bazaar, The Beaches West
20
Commerce Court, Victoria Hotel
100
Studio District
41
Lawrence Park
3
Roselawn
3
Davisville North
7
Forest Hill West, Forest Hill North
4
The Junction South, High Park
24
North Toronto West
20
North Midtown, The Annex, Yorkville
22
Roncesvalles, Parkdale
13
Davisville
36
Harbord, University of Toronto
37
Swansea, Runnymede
37
Moore Park, Summerhill East
3
Chinatown, Kensington Market, Grange Park
89
Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill
16
Railway Lands, South Niagara, Harbourfront West, CN Tower, King and Spadina, Bathurs

In [35]:
Toronto_venues['Neighborhood'].value_counts()

Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Garden District, Ryerson                                                                                      100
Richmond, King, Adelaide                                                                                      100
Union Station, Toronto Islands, Harbourfront East                                                             100
Toronto Dominion Centre, Design Exchange                                                                      100
St. James Town                                                                                                100
Stn A PO Boxes 25 The Esplanade                                                                                97
Chinatown, Kensington Market, Grange Park                                               

# Analyzing each neighborhood

In [36]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
Toronto_onehot.set_index('Neighborhood',inplace=True)
Toronto_onehot.reset_index(drop=False,inplace=True)
Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.012195,0.000000,...,0.000000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.00,0.012195
4,"Chinatown, Kensington Market, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.056180,0.000000,0.056180,0.011236,0.000000,0.00,0.000000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.011765,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.011765,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.011765,0.000000,0.011765,0.00,0.023529
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.030000,0.000000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [38]:
Toronto_grouped.shape

(38, 237)

# Cluster neighborhoods

In [41]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
# adding cluster labels
Toronto_clusters=pd.DataFrame(Toronto_grouped['Neighborhood'])
Toronto_clusters.insert(0,'Cluster Labels',kmeans.labels_)
Toronto_merged = Toronto_data
Toronto_merged=Toronto_merged.join(Toronto_clusters.set_index('Neighborhood'),on='Neighborhood')
Toronto_merged.dropna(inplace=True)
Toronto_merged.reset_index(drop=True,inplace=True)

In [43]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters